In [1]:
# drop_from_seohee_robust.py  (노트북 셀에 붙여넣어 실행)
from pathlib import Path
import pandas as pd

# 1) 파일명(품목군)별 드랍할 "세부 품목" 규칙  ─ ❌ + ⚠️ 전부 포함
RULES = {
    "당근": {"당근(수입)"},
    "대파": {"대파(수입)", "깐대파"},
    "양파": {"기타", "양파(수입)", "양파(일반)", "저장양파", "조생양파", "중생양파", "포장양파"},
    "피망전체": {"반홍피망", "기타"},
    "홍고추": set(),  # 모두 사용 가능 → 드랍 없음
}
# 여기 명시되지 않은 파일명(파프리카, 청고추, 마늘 등)은 "건드리지 않음"

CANDIDATE_COLS = ["품목명", "세부 품목", "세부품목", "품목"]  # 세부품목 컬럼 후보

# 2) 루트 폴더 자동 탐색 (현재/상하위/홈 고정 경로)
def pick_root():
    cwd = Path.cwd().resolve()
    candidates = [
        cwd,
        cwd / "23 서희",
        cwd.parent / "23 서희",
        Path.home() / "Documents/GitHub/GDF_Final_G3/sojin/23 서희",
    ]
    for p in candidates:
        if p.exists() and p.is_dir() and list(p.glob("*.csv")):
            return p.resolve()
    # 그래도 못 찾으면 트리 검색
    hits = [h for h in cwd.glob("**/23 서희") if list(h.glob("*.csv"))]
    if hits:
        return hits[0].resolve()
    raise SystemExit(f"[에러] '23 서희' 폴더를 찾지 못했습니다. CWD={cwd}")

root = pick_root()
print(f"[INFO] 작업 폴더: {root}")

# 3) 처리 대상: RULES에 있는 파일명만
target_files = []
for p in sorted(root.glob("*.csv")):
    stem = p.stem.strip()
    if stem in RULES and not p.name.endswith("_cleaned.csv"):
        target_files.append(p)

print(f"[INFO] 처리 대상 CSV: {len(target_files)}개 -> {[p.stem for p in target_files]}")
if not target_files:
    raise SystemExit("[중지] RULES에 해당하는 파일이 없습니다.")

total_before = total_after = 0

for fp in target_files:
    stem = fp.stem.strip()     # 당근/대파/양파/피망전체/홍고추
    drop_items = RULES.get(stem, set())

    # 4) 읽기 (인코딩 자동)
    try:
        df = pd.read_csv(fp, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="cp949")

    # 5) 공백 정리
    df.columns = df.columns.str.strip()
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()

    # 6) 세부품목 컬럼 찾기
    target_col = next((c for c in CANDIDATE_COLS if c in df.columns), None)
    if target_col is None:
        print(f"[SKIP] {fp.name}: 세부품목 컬럼을 찾지 못함 -> 보유칼럼: {list(df.columns)[:10]}")
        continue

    before = len(df)
    present = sorted(set(df[target_col]) & drop_items)
    df2 = df[~df[target_col].isin(drop_items)].copy()
    after = len(df2)

    # 7) 저장 (원본 옆 cleaned/)
    out_dir = fp.parent / "cleaned"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{fp.stem}_cleaned.csv"
    df2.to_csv(out_path, index=False, encoding="utf-8-sig")

    print(f"[OK] {fp.name}: {before} → {after} (드랍 {before-after}행) | 사용컬럼: {target_col} "
          f"| 제거된 세부품목: {present or '-'} | 저장: {out_path.name}")

    total_before += before
    total_after += after

print(f"\n[요약] {len(target_files)}개 처리 완료. 총 {total_before}행 → {total_after}행")
print(f"[완료] 결과는 '{root}/cleaned' 폴더에서 확인하세요.")


[INFO] 작업 폴더: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/23 서희
[INFO] 처리 대상 CSV: 4개 -> ['당근', '양파', '피망전체', '홍고추']
[OK] 당근.csv: 7308 → 3654 (드랍 3654행) | 사용컬럼: 품목명 | 제거된 세부품목: ['당근(수입)'] | 저장: 당근_cleaned.csv
[OK] 양파.csv: 36540 → 10962 (드랍 25578행) | 사용컬럼: 품목명 | 제거된 세부품목: ['기타', '양파(수입)', '양파(일반)', '저장양파', '조생양파', '중생양파', '포장양파'] | 저장: 양파_cleaned.csv
[OK] 피망전체.csv: 14616 → 7308 (드랍 7308행) | 사용컬럼: 품목명 | 제거된 세부품목: ['기타', '반홍피망'] | 저장: 피망전체_cleaned.csv
[OK] 홍고추.csv: 7308 → 7308 (드랍 0행) | 사용컬럼: 품목명 | 제거된 세부품목: - | 저장: 홍고추_cleaned.csv

[요약] 4개 처리 완료. 총 65772행 → 29232행
[완료] 결과는 '/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/23 서희/cleaned' 폴더에서 확인하세요.
